# 02 - Data Processing

In [1]:
import sys
sys.executable

'/usr/local/bin/python'

## Imports

In [2]:
import numpy as np
import pandas as pd

import constants as cnst
import stock_utils as su

pd.set_option('display.max_columns', None)

NSE_DATA_DIR = PosixPath('../data/NSE') | Valid: True
PROCESSED_DATA_DIR = PosixPath('../data/processed') | Valid: True


## Constants

In [3]:
stock_symbols = su.get_all_stock_symbols(
    cnst.NSE_DATA_DIR
)

stock_symbols

['DEEPAKFERT', 'HDFCBANK', 'ITBEES', 'JUBLFOOD']

In [4]:
STOCK_SYMBOL = stock_symbols[0]
STOCK_SYMBOL

'DEEPAKFERT'

## Data loading

In [5]:
stock_df = pd.read_parquet(cnst.NSE_DATA_DIR.joinpath(STOCK_SYMBOL, "consolidated.parquet"))
stock_df = stock_df.drop(columns = ['series', 'PREV. CLOSE'])
stock_df.columns = [
    'Date',
    'Open', 'High', 'Low', 'LTP', 'Close',
    'VWAP', '52W H', '52W L',
    'Volume', 'Value', '#Trades'
]
stock_df

,Date,Open,High,Low,LTP,Close,VWAP,52W H,52W L,Volume,Value,#Trades
0,2020-01-01,97.20,97.85,96.30,96.60,96.75,96.95,169.0,76.20,153261,1.485918e+07,3161
1,2020-01-02,96.85,102.40,96.60,100.00,100.35,100.17,169.0,76.20,637039,6.381519e+07,7526
2,2020-01-03,99.70,104.80,99.00,101.20,100.75,101.90,169.0,76.20,562843,5.735485e+07,7402
3,2020-01-06,100.50,100.85,95.25,95.70,96.10,97.91,169.0,76.20,326011,3.192018e+07,5214
4,2020-01-07,97.35,101.00,97.10,99.95,100.40,99.09,169.0,76.20,307107,3.043237e+07,4983
...,...,...,...,...,...,...,...,...,...,...,...,...
976,2024-02-12,515.50,517.10,497.85,502.80,500.05,506.51,714.7,497.85,654659,3.315936e+08,26328
977,2024-02-13,500.10,503.80,483.35,498.95,495.05,493.89,714.7,483.35,827799,4.088438e+08,39023
978,2024-02-14,495.00,499.70,488.50,490.25,490.80,493.16,714.7,483.35,633883,3.126049e+08,31700
979,2024-02-15,493.50,515.30,491.95,508.10,507.90,508.02,714.7,483.35,995334,5.056480e+08,49286


In [6]:
stock_df.info(memory_usage = "deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 981 entries, 0 to 980
Data columns (total 12 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   Date     981 non-null    datetime64[ns]
 1   Open     981 non-null    float64       
 2   High     981 non-null    float64       
 3   Low      981 non-null    float64       
 4   LTP      981 non-null    float64       
 5   Close    981 non-null    float64       
 6   VWAP     981 non-null    float64       
 7   52W H    981 non-null    float64       
 8   52W L    981 non-null    float64       
 9   Volume   981 non-null    int64         
 10  Value    981 non-null    float64       
 11  #Trades  981 non-null    int64         
dtypes: datetime64[ns](1), float64(9), int64(2)
memory usage: 92.1 KB


## Feature engineering

### Daily candle based columns

In [7]:
stock_df['Range'] = stock_df['High'] - stock_df['Low']
stock_df['IsGreen'] = (stock_df['Close'] >= stock_df['Open']).astype(int)
stock_df['Is52WLow'] = np.isclose(stock_df['Low'], stock_df['52W L'], atol = 0, rtol = 1e-4).astype(int)
stock_df['Is52WHigh'] = np.isclose(stock_df['High'], stock_df['52W H'], atol = 0, rtol = 1e-4).astype(int)
stock_df.sample(10)

,Date,Open,High,Low,LTP,Close,VWAP,52W H,52W L,Volume,Value,#Trades,Range,IsGreen,Is52WLow,Is52WHigh
500,2022-03-11,554.80,573.50,552.10,557.0,558.40,563.86,659.90,199.35,834098,4.703126e+08,15580,21.40,1,0,0
612,2022-08-24,989.45,996.85,942.00,951.0,954.40,963.15,996.85,343.55,733073,7.060609e+08,33641,54.85,0,0,1
738,2023-02-23,629.15,634.80,620.00,625.5,628.15,628.56,1062.00,506.65,196577,1.235610e+08,11385,14.80,0,0,0
653,2022-10-24,1042.00,1044.60,1004.00,1016.6,1016.50,1021.47,1062.00,343.55,241845,2.470372e+08,11226,40.60,0,0,0
584,2022-07-13,660.10,667.80,635.50,641.4,639.90,650.97,724.80,343.55,254510,1.656794e+08,11114,32.30,0,0,0
541,2022-05-13,557.00,568.70,529.80,538.8,535.05,552.31,724.80,267.35,284378,1.570639e+08,10178,38.90,0,0,0
269,2021-01-04,159.20,160.00,157.25,158.0,157.75,158.11,193.00,57.15,343543,5.431790e+07,3692,2.75,0,0,0
186,2020-09-04,170.00,174.70,168.70,170.8,170.20,171.60,193.00,57.15,724993,1.244089e+08,11417,6.00,1,0,0
144,2020-07-08,119.70,122.95,118.50,119.7,119.35,120.90,126.65,57.15,2095224,2.533037e+08,21613,4.45,0,0,0
124,2020-06-10,105.40,106.50,104.30,105.0,105.20,105.16,143.00,57.15,322554,3.391968e+07,3615,2.20,0,0,0


In [8]:
((stock_df[['IsGreen', 'Is52WLow', 'Is52WHigh']].sum() * 100) / len(stock_df)).round(2)

IsGreen      41.79
Is52WLow      0.61
Is52WHigh     5.50
dtype: float64

In [9]:
stock_df.shape

(981, 16)

### Date based columns

In [10]:
stock_df['Day'] = stock_df['Date'].dt.day
stock_df['Month'] = stock_df['Date'].dt.month
stock_df['Year'] = stock_df['Date'].dt.year
stock_df['Weekday'] = stock_df['Date'].dt.weekday
stock_df['DayOfYear'] = stock_df['Date'].dt.day_of_year
stock_df['Quarter'] = stock_df['Date'].dt.quarter
stock_df['DaysSinceLastTradingSession'] = (stock_df['Date'] - stock_df['Date'].shift(1)).dt.days.fillna(1).astype(int)

stock_df.sample(10)

,Date,Open,High,Low,LTP,Close,VWAP,52W H,52W L,Volume,Value,#Trades,Range,IsGreen,Is52WLow,Is52WHigh,Day,Month,Year,Weekday,DayOfYear,Quarter,DaysSinceLastTradingSession
118,2020-06-02,106.0,106.40,102.85,104.0,103.85,105.00,146.00,57.15,682352,7.164776e+07,4729,3.55,0,0,0,2,6,2020,1,154,2,1
436,2021-12-09,370.1,373.75,364.15,365.0,366.70,369.34,474.50,144.50,89723,3.313818e+07,2727,9.60,0,0,0,9,12,2021,3,343,4,1
528,2022-04-25,696.0,707.45,675.00,680.4,678.55,691.52,724.80,236.00,368311,2.546962e+08,9521,32.45,0,0,0,25,4,2022,0,115,2,3
227,2020-11-03,152.0,157.70,146.75,146.8,148.40,152.76,193.00,57.15,2995329,4.575591e+08,31681,10.95,0,0,0,3,11,2020,1,308,4,1
649,2022-10-18,978.0,1024.00,955.55,967.5,965.45,994.52,1024.00,343.55,2787026,2.771759e+09,93302,68.45,0,0,1,18,10,2022,1,291,4,1
251,2020-12-08,162.9,170.60,161.55,169.0,168.95,166.70,193.00,57.15,2770162,4.617972e+08,27741,9.05,1,0,0,8,12,2020,1,343,4,1
963,2024-01-23,658.0,658.00,625.00,627.1,628.90,636.69,714.70,512.40,485414,3.090559e+08,23010,33.00,0,0,0,23,1,2024,1,23,1,3
776,2023-04-25,595.9,610.50,592.60,607.6,606.10,601.99,1062.00,512.40,384413,2.314130e+08,11215,17.90,1,0,0,25,4,2023,1,115,2,1
546,2022-05-20,572.5,576.60,563.05,575.0,571.50,570.20,724.80,278.00,168349,9.599282e+07,5135,13.55,0,0,0,20,5,2022,4,140,2,1
346,2021-04-29,261.8,269.20,253.10,264.6,263.65,262.76,276.45,95.15,618296,1.624653e+08,13031,16.10,1,0,0,29,4,2021,3,119,2,1


In [11]:
stock_df.shape

(981, 23)

### Moving averages

#### `Close`

In [12]:
for window in cnst.ROLLING_WINDOWS:
    stock_df[f'Close {window}MA'] = stock_df['Close'].rolling(window = window, min_periods = 1).mean().round(2)

stock_df.filter(regex = "Close.*")

,Close,Close 7MA,Close 15MA,Close 30MA,Close 60MA
0,96.75,96.75,96.75,96.75,96.75
1,100.35,98.55,98.55,98.55,98.55
2,100.75,99.28,99.28,99.28,99.28
3,96.10,98.49,98.49,98.49,98.49
4,100.40,98.87,98.87,98.87,98.87
...,...,...,...,...,...
976,500.05,532.51,581.46,623.67,634.67
977,495.05,521.22,571.23,617.25,632.67
978,490.80,514.09,562.03,610.53,630.67
979,507.90,509.79,553.37,604.57,629.15


#### `Range`

In [13]:
for window in cnst.ROLLING_WINDOWS:
    stock_df[f'Range {window}MA'] = stock_df['Range'].rolling(window = window, min_periods = 1).mean().round(2)

stock_df.filter(regex = "Range.*")

,Range,Range 7MA,Range 15MA,Range 30MA,Range 60MA
0,1.55,1.55,1.55,1.55,1.55
1,5.80,3.68,3.68,3.68,3.68
2,5.80,4.38,4.38,4.38,4.38
3,5.60,4.69,4.69,4.69,4.69
4,3.90,4.53,4.53,4.53,4.53
...,...,...,...,...,...
976,19.25,22.49,23.74,21.57,21.98
977,20.45,21.92,23.85,21.44,22.16
978,11.20,17.86,22.40,21.00,22.15
979,23.35,18.63,22.59,21.34,22.26


#### `VWAP`

In [14]:
for window in cnst.ROLLING_WINDOWS:
    stock_df[f'VWAP {window}MA'] = stock_df['VWAP'].rolling(window = window, min_periods = 1).mean().round(2)

stock_df.filter(regex = "VWAP.*")

,VWAP,VWAP 7MA,VWAP 15MA,VWAP 30MA,VWAP 60MA
0,96.95,96.95,96.95,96.95,96.95
1,100.17,98.56,98.56,98.56,98.56
2,101.90,99.67,99.67,99.67,99.67
3,97.91,99.23,99.23,99.23,99.23
4,99.09,99.20,99.20,99.20,99.20
...,...,...,...,...,...
976,506.51,538.22,586.38,627.50,637.57
977,493.89,525.74,575.63,620.93,635.50
978,493.16,517.33,566.06,614.19,633.49
979,508.02,512.59,557.76,608.17,631.92


#### `Volume`

In [15]:
for window in cnst.ROLLING_WINDOWS:
    stock_df[f'Volume {window}MA'] = stock_df['Volume'].rolling(window = window, min_periods = 1).mean().astype(int)

stock_df.filter(regex = "Volume.*")

,Volume,Volume 7MA,Volume 15MA,Volume 30MA,Volume 60MA
0,153261,153261,153261,153261,153261
1,637039,395150,395150,395150,395150
2,562843,451047,451047,451047,451047
3,326011,419788,419788,419788,419788
4,307107,397252,397252,397252,397252
...,...,...,...,...,...
976,654659,928983,781001,679848,729576
977,827799,858821,808121,657226,739972
978,633883,735754,818019,640038,747111
979,995334,711399,860316,656385,757426


#### `Value`

In [16]:
for window in cnst.ROLLING_WINDOWS:
    stock_df[f'Value {window}MA'] = stock_df['Value'].rolling(window = window, min_periods = 1).mean().astype(int)

stock_df.filter(regex = "Value.*")

,Value,Value 7MA,Value 15MA,Value 30MA,Value 60MA
0,1.485918e+07,14859179,14859179,14859179,14859179
1,6.381519e+07,39337186,39337186,39337186,39337186
2,5.735485e+07,45343075,45343075,45343075,45343075
3,3.192018e+07,41987352,41987352,41987352,41987352
4,3.043237e+07,39676356,39676356,39676356,39676356
...,...,...,...,...,...
976,3.315936e+08,506187770,450855191,420705230,467952521
977,4.088438e+08,455081470,459721553,399636888,472665722
978,3.126049e+08,381813171,459958157,383406355,475772685
979,5.056480e+08,363912813,478450515,388672600,480421170


#### `#Trades`

In [17]:
for window in cnst.ROLLING_WINDOWS:
    stock_df[f'#Trades {window}MA'] = stock_df['#Trades'].rolling(window = window, min_periods = 1).mean().astype(int)

stock_df.filter(regex = "#Trades.*")

,#Trades,#Trades 7MA,#Trades 15MA,#Trades 30MA,#Trades 60MA
0,3161,3161,3161,3161,3161
1,7526,5343,5343,5343,5343
2,7402,6029,6029,6029,6029
3,5214,5825,5825,5825,5825
4,4983,5657,5657,5657,5657
...,...,...,...,...,...
976,26328,35476,29835,24604,24325
977,39023,34396,31390,24655,24855
978,31700,30659,31969,24742,25240
979,49286,32106,34057,25877,25807


### Target columns

In [18]:
for window in cnst.TARGET_WINDOWS:
    stock_df[f'Target {window}D'] = stock_df['Close'].shift(-window)

stock_df.filter(regex = "Target.*")

,Target 3D,Target 7D,Target 15D,Target 30D
0,96.10,106.20,109.70,107.00
1,100.40,109.65,111.55,103.30
2,98.95,115.30,114.85,99.45
3,106.30,119.85,121.25,99.95
4,106.20,118.20,116.60,96.55
...,...,...,...,...
976,507.90,NaN,NaN,NaN
977,494.50,NaN,NaN,NaN
978,NaN,NaN,NaN,NaN
979,NaN,NaN,NaN,NaN


### Saving processed data

In [19]:
stock_df.to_parquet(
    cnst.PROCESSED_DATA_DIR.joinpath(f'{STOCK_SYMBOL}-processed.parquet'), 
    index = False
)
stock_df

,Date,Open,High,Low,LTP,Close,VWAP,52W H,52W L,Volume,Value,#Trades,Range,IsGreen,Is52WLow,Is52WHigh,Day,Month,Year,Weekday,DayOfYear,Quarter,DaysSinceLastTradingSession,Close 7MA,Close 15MA,Close 30MA,Close 60MA,Range 7MA,Range 15MA,Range 30MA,Range 60MA,VWAP 7MA,VWAP 15MA,VWAP 30MA,VWAP 60MA,Volume 7MA,Volume 15MA,Volume 30MA,Volume 60MA,Value 7MA,Value 15MA,Value 30MA,Value 60MA,#Trades 7MA,#Trades 15MA,#Trades 30MA,#Trades 60MA,Target 3D,Target 7D,Target 15D,Target 30D
0,2020-01-01,97.20,97.85,96.30,96.60,96.75,96.95,169.0,76.20,153261,1.485918e+07,3161,1.55,0,0,0,1,1,2020,2,1,1,1,96.75,96.75,96.75,96.75,1.55,1.55,1.55,1.55,96.95,96.95,96.95,96.95,153261,153261,153261,153261,14859179,14859179,14859179,14859179,3161,3161,3161,3161,96.10,106.20,109.70,107.00
1,2020-01-02,96.85,102.40,96.60,100.00,100.35,100.17,169.0,76.20,637039,6.381519e+07,7526,5.80,1,0,0,2,1,2020,3,2,1,1,98.55,98.55,98.55,98.55,3.68,3.68,3.68,3.68,98.56,98.56,98.56,98.56,395150,395150,395150,395150,39337186,39337186,39337186,39337186,5343,5343,5343,5343,100.40,109.65,111.55,103.30
2,2020-01-03,99.70,104.80,99.00,101.20,100.75,101.90,169.0,76.20,562843,5.735485e+07,7402,5.80,1,0,0,3,1,2020,4,3,1,1,99.28,99.28,99.28,99.28,4.38,4.38,4.38,4.38,99.67,99.67,99.67,99.67,451047,451047,451047,451047,45343075,45343075,45343075,45343075,6029,6029,6029,6029,98.95,115.30,114.85,99.45
3,2020-01-06,100.50,100.85,95.25,95.70,96.10,97.91,169.0,76.20,326011,3.192018e+07,5214,5.60,0,0,0,6,1,2020,0,6,1,3,98.49,98.49,98.49,98.49,4.69,4.69,4.69,4.69,99.23,99.23,99.23,99.23,419788,419788,419788,419788,41987352,41987352,41987352,41987352,5825,5825,5825,5825,106.30,119.85,121.25,99.95
4,2020-01-07,97.35,101.00,97.10,99.95,100.40,99.09,169.0,76.20,307107,3.043237e+07,4983,3.90,1,0,0,7,1,2020,1,7,1,1,98.87,98.87,98.87,98.87,4.53,4.53,4.53,4.53,99.20,99.20,99.20,99.20,397252,397252,397252,397252,39676356,39676356,39676356,39676356,5657,5657,5657,5657,106.20,118.20,116.60,96.55
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
976,2024-02-12,515.50,517.10,497.85,502.80,500.05,506.51,714.7,497.85,654659,3.315936e+08,26328,19.25,0,1,0,12,2,2024,0,43,1,3,532.51,581.46,623.67,634.67,22.49,23.74,21.57,21.98,538.22,586.38,627.50,637.57,928983,781001,679848,729576,506187770,450855191,420705230,467952521,35476,29835,24604,24325,507.90,NaN,NaN,NaN
977,2024-02-13,500.10,503.80,483.35,498.95,495.05,493.89,714.7,483.35,827799,4.088438e+08,39023,20.45,0,1,0,13,2,2024,1,44,1,1,521.22,571.23,617.25,632.67,21.92,23.85,21.44,22.16,525.74,575.63,620.93,635.50,858821,808121,657226,739972,455081470,459721553,399636888,472665722,34396,31390,24655,24855,494.50,NaN,NaN,NaN
978,2024-02-14,495.00,499.70,488.50,490.25,490.80,493.16,714.7,483.35,633883,3.126049e+08,31700,11.20,0,0,0,14,2,2024,2,45,1,1,514.09,562.03,610.53,630.67,17.86,22.40,21.00,22.15,517.33,566.06,614.19,633.49,735754,818019,640038,747111,381813171,459958157,383406355,475772685,30659,31969,24742,25240,NaN,NaN,NaN,NaN
979,2024-02-15,493.50,515.30,491.95,508.10,507.90,508.02,714.7,483.35,995334,5.056480e+08,49286,23.35,1,0,0,15,2,2024,3,46,1,1,509.79,553.37,604.57,629.15,18.63,22.59,21.34,22.26,512.59,557.76,608.17,631.92,711399,860316,656385,757426,363912813,478450515,388672600,480421170,32106,34057,25877,25807,NaN,NaN,NaN,NaN


## Data standardization

### Stock price based columns

In [20]:
stock_price_cols = [
    'Open', 'High', 'Low', 'LTP', '52W H', '52W L'
] + (
    stock_df.filter(regex = "Close.*").columns.to_list() +
    stock_df.filter(regex = "Range.*").columns.to_list() +
    stock_df.filter(regex = "VWAP.*").columns.to_list() +
    stock_df.filter(regex = "Target.*").columns.to_list()
)

stock_price_cols

['Open',
 'High',
 'Low',
 'LTP',
 '52W H',
 '52W L',
 'Close',
 'Close 7MA',
 'Close 15MA',
 'Close 30MA',
 'Close 60MA',
 'Range',
 'Range 7MA',
 'Range 15MA',
 'Range 30MA',
 'Range 60MA',
 'VWAP',
 'VWAP 7MA',
 'VWAP 15MA',
 'VWAP 30MA',
 'VWAP 60MA',
 'Target 3D',
 'Target 7D',
 'Target 15D',
 'Target 30D']

In [21]:
stock_df[stock_price_cols] = stock_df[stock_price_cols].div(stock_df['Close'], axis = 0).round(3)
stock_df[stock_price_cols]

,Open,High,Low,LTP,52W H,52W L,Close,Close 7MA,Close 15MA,Close 30MA,Close 60MA,Range,Range 7MA,Range 15MA,Range 30MA,Range 60MA,VWAP,VWAP 7MA,VWAP 15MA,VWAP 30MA,VWAP 60MA,Target 3D,Target 7D,Target 15D,Target 30D
0,1.005,1.011,0.995,0.998,1.747,0.788,1.0,1.000,1.000,1.000,1.000,0.016,0.016,0.016,0.016,0.016,1.002,1.002,1.002,1.002,1.002,0.993,1.098,1.134,1.106
1,0.965,1.020,0.963,0.997,1.684,0.759,1.0,0.982,0.982,0.982,0.982,0.058,0.037,0.037,0.037,0.037,0.998,0.982,0.982,0.982,0.982,1.000,1.093,1.112,1.029
2,0.990,1.040,0.983,1.004,1.677,0.756,1.0,0.985,0.985,0.985,0.985,0.058,0.043,0.043,0.043,0.043,1.011,0.989,0.989,0.989,0.989,0.982,1.144,1.140,0.987
3,1.046,1.049,0.991,0.996,1.759,0.793,1.0,1.025,1.025,1.025,1.025,0.058,0.049,0.049,0.049,0.049,1.019,1.033,1.033,1.033,1.033,1.106,1.247,1.262,1.040
4,0.970,1.006,0.967,0.996,1.683,0.759,1.0,0.985,0.985,0.985,0.985,0.039,0.045,0.045,0.045,0.045,0.987,0.988,0.988,0.988,0.988,1.058,1.177,1.161,0.962
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
976,1.031,1.034,0.996,1.005,1.429,0.996,1.0,1.065,1.163,1.247,1.269,0.038,0.045,0.047,0.043,0.044,1.013,1.076,1.173,1.255,1.275,1.016,NaN,NaN,NaN
977,1.010,1.018,0.976,1.008,1.444,0.976,1.0,1.053,1.154,1.247,1.278,0.041,0.044,0.048,0.043,0.045,0.998,1.062,1.163,1.254,1.284,0.999,NaN,NaN,NaN
978,1.009,1.018,0.995,0.999,1.456,0.985,1.0,1.047,1.145,1.244,1.285,0.023,0.036,0.046,0.043,0.045,1.005,1.054,1.153,1.251,1.291,NaN,NaN,NaN,NaN
979,0.972,1.015,0.969,1.000,1.407,0.952,1.0,1.004,1.090,1.190,1.239,0.046,0.037,0.044,0.042,0.044,1.000,1.009,1.098,1.197,1.244,NaN,NaN,NaN,NaN


### Volume based columns

In [22]:
volume_cols = stock_df.filter(regex = "Volume.*").columns.to_list()

volume_cols

['Volume', 'Volume 7MA', 'Volume 15MA', 'Volume 30MA', 'Volume 60MA']

In [23]:
stock_df[volume_cols] = stock_df[volume_cols].div(stock_df['Volume'], axis = 0).round(3)
stock_df[volume_cols]

,Volume,Volume 7MA,Volume 15MA,Volume 30MA,Volume 60MA
0,1.0,1.000,1.000,1.000,1.000
1,1.0,0.620,0.620,0.620,0.620
2,1.0,0.801,0.801,0.801,0.801
3,1.0,1.288,1.288,1.288,1.288
4,1.0,1.294,1.294,1.294,1.294
...,...,...,...,...,...
976,1.0,1.419,1.193,1.038,1.114
977,1.0,1.037,0.976,0.794,0.894
978,1.0,1.161,1.290,1.010,1.179
979,1.0,0.715,0.864,0.659,0.761


### Value based columns

In [24]:
value_cols = stock_df.filter(regex = "Value.*").columns.to_list()

value_cols

['Value', 'Value 7MA', 'Value 15MA', 'Value 30MA', 'Value 60MA']

In [25]:
stock_df[value_cols] = stock_df[value_cols].div(stock_df['Value'], axis = 0).round(3)
stock_df[value_cols]

,Value,Value 7MA,Value 15MA,Value 30MA,Value 60MA
0,1.0,1.000,1.000,1.000,1.000
1,1.0,0.616,0.616,0.616,0.616
2,1.0,0.791,0.791,0.791,0.791
3,1.0,1.315,1.315,1.315,1.315
4,1.0,1.304,1.304,1.304,1.304
...,...,...,...,...,...
976,1.0,1.527,1.360,1.269,1.411
977,1.0,1.113,1.124,0.977,1.156
978,1.0,1.221,1.471,1.226,1.522
979,1.0,0.720,0.946,0.769,0.950


### Trade count based columns

In [26]:
trade_count_cols = stock_df.filter(regex = "#Trades.*").columns.to_list()

trade_count_cols

['#Trades', '#Trades 7MA', '#Trades 15MA', '#Trades 30MA', '#Trades 60MA']

In [27]:
stock_df[trade_count_cols] = stock_df[trade_count_cols].div(stock_df['#Trades'], axis = 0).round(3)
stock_df[trade_count_cols]

,#Trades,#Trades 7MA,#Trades 15MA,#Trades 30MA,#Trades 60MA
0,1.0,1.000,1.000,1.000,1.000
1,1.0,0.710,0.710,0.710,0.710
2,1.0,0.815,0.815,0.815,0.815
3,1.0,1.117,1.117,1.117,1.117
4,1.0,1.135,1.135,1.135,1.135
...,...,...,...,...,...
976,1.0,1.347,1.133,0.935,0.924
977,1.0,0.881,0.804,0.632,0.637
978,1.0,0.967,1.008,0.781,0.796
979,1.0,0.651,0.691,0.525,0.524


### Dropping unnecessary columns

In [28]:
stock_df = stock_df.drop(columns = ['Date', 'Close', 'Range', 'Volume', 'Value', '#Trades'])
stock_df

,Open,High,Low,LTP,VWAP,52W H,52W L,IsGreen,Is52WLow,Is52WHigh,Day,Month,Year,Weekday,DayOfYear,Quarter,DaysSinceLastTradingSession,Close 7MA,Close 15MA,Close 30MA,Close 60MA,Range 7MA,Range 15MA,Range 30MA,Range 60MA,VWAP 7MA,VWAP 15MA,VWAP 30MA,VWAP 60MA,Volume 7MA,Volume 15MA,Volume 30MA,Volume 60MA,Value 7MA,Value 15MA,Value 30MA,Value 60MA,#Trades 7MA,#Trades 15MA,#Trades 30MA,#Trades 60MA,Target 3D,Target 7D,Target 15D,Target 30D
0,1.005,1.011,0.995,0.998,1.002,1.747,0.788,0,0,0,1,1,2020,2,1,1,1,1.000,1.000,1.000,1.000,0.016,0.016,0.016,0.016,1.002,1.002,1.002,1.002,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,0.993,1.098,1.134,1.106
1,0.965,1.020,0.963,0.997,0.998,1.684,0.759,1,0,0,2,1,2020,3,2,1,1,0.982,0.982,0.982,0.982,0.037,0.037,0.037,0.037,0.982,0.982,0.982,0.982,0.620,0.620,0.620,0.620,0.616,0.616,0.616,0.616,0.710,0.710,0.710,0.710,1.000,1.093,1.112,1.029
2,0.990,1.040,0.983,1.004,1.011,1.677,0.756,1,0,0,3,1,2020,4,3,1,1,0.985,0.985,0.985,0.985,0.043,0.043,0.043,0.043,0.989,0.989,0.989,0.989,0.801,0.801,0.801,0.801,0.791,0.791,0.791,0.791,0.815,0.815,0.815,0.815,0.982,1.144,1.140,0.987
3,1.046,1.049,0.991,0.996,1.019,1.759,0.793,0,0,0,6,1,2020,0,6,1,3,1.025,1.025,1.025,1.025,0.049,0.049,0.049,0.049,1.033,1.033,1.033,1.033,1.288,1.288,1.288,1.288,1.315,1.315,1.315,1.315,1.117,1.117,1.117,1.117,1.106,1.247,1.262,1.040
4,0.970,1.006,0.967,0.996,0.987,1.683,0.759,1,0,0,7,1,2020,1,7,1,1,0.985,0.985,0.985,0.985,0.045,0.045,0.045,0.045,0.988,0.988,0.988,0.988,1.294,1.294,1.294,1.294,1.304,1.304,1.304,1.304,1.135,1.135,1.135,1.135,1.058,1.177,1.161,0.962
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
976,1.031,1.034,0.996,1.005,1.013,1.429,0.996,0,1,0,12,2,2024,0,43,1,3,1.065,1.163,1.247,1.269,0.045,0.047,0.043,0.044,1.076,1.173,1.255,1.275,1.419,1.193,1.038,1.114,1.527,1.360,1.269,1.411,1.347,1.133,0.935,0.924,1.016,NaN,NaN,NaN
977,1.010,1.018,0.976,1.008,0.998,1.444,0.976,0,1,0,13,2,2024,1,44,1,1,1.053,1.154,1.247,1.278,0.044,0.048,0.043,0.045,1.062,1.163,1.254,1.284,1.037,0.976,0.794,0.894,1.113,1.124,0.977,1.156,0.881,0.804,0.632,0.637,0.999,NaN,NaN,NaN
978,1.009,1.018,0.995,0.999,1.005,1.456,0.985,0,0,0,14,2,2024,2,45,1,1,1.047,1.145,1.244,1.285,0.036,0.046,0.043,0.045,1.054,1.153,1.251,1.291,1.161,1.290,1.010,1.179,1.221,1.471,1.226,1.522,0.967,1.008,0.781,0.796,NaN,NaN,NaN,NaN
979,0.972,1.015,0.969,1.000,1.000,1.407,0.952,1,0,0,15,2,2024,3,46,1,1,1.004,1.090,1.190,1.239,0.037,0.044,0.042,0.044,1.009,1.098,1.197,1.244,0.715,0.864,0.659,0.761,0.720,0.946,0.769,0.950,0.651,0.691,0.525,0.524,NaN,NaN,NaN,NaN


### Saving standardized data

In [29]:
stock_df.to_parquet(
    cnst.PROCESSED_DATA_DIR.joinpath(f'{STOCK_SYMBOL}-standardized.parquet'), 
    index = False
)
stock_df

,Open,High,Low,LTP,VWAP,52W H,52W L,IsGreen,Is52WLow,Is52WHigh,Day,Month,Year,Weekday,DayOfYear,Quarter,DaysSinceLastTradingSession,Close 7MA,Close 15MA,Close 30MA,Close 60MA,Range 7MA,Range 15MA,Range 30MA,Range 60MA,VWAP 7MA,VWAP 15MA,VWAP 30MA,VWAP 60MA,Volume 7MA,Volume 15MA,Volume 30MA,Volume 60MA,Value 7MA,Value 15MA,Value 30MA,Value 60MA,#Trades 7MA,#Trades 15MA,#Trades 30MA,#Trades 60MA,Target 3D,Target 7D,Target 15D,Target 30D
0,1.005,1.011,0.995,0.998,1.002,1.747,0.788,0,0,0,1,1,2020,2,1,1,1,1.000,1.000,1.000,1.000,0.016,0.016,0.016,0.016,1.002,1.002,1.002,1.002,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,0.993,1.098,1.134,1.106
1,0.965,1.020,0.963,0.997,0.998,1.684,0.759,1,0,0,2,1,2020,3,2,1,1,0.982,0.982,0.982,0.982,0.037,0.037,0.037,0.037,0.982,0.982,0.982,0.982,0.620,0.620,0.620,0.620,0.616,0.616,0.616,0.616,0.710,0.710,0.710,0.710,1.000,1.093,1.112,1.029
2,0.990,1.040,0.983,1.004,1.011,1.677,0.756,1,0,0,3,1,2020,4,3,1,1,0.985,0.985,0.985,0.985,0.043,0.043,0.043,0.043,0.989,0.989,0.989,0.989,0.801,0.801,0.801,0.801,0.791,0.791,0.791,0.791,0.815,0.815,0.815,0.815,0.982,1.144,1.140,0.987
3,1.046,1.049,0.991,0.996,1.019,1.759,0.793,0,0,0,6,1,2020,0,6,1,3,1.025,1.025,1.025,1.025,0.049,0.049,0.049,0.049,1.033,1.033,1.033,1.033,1.288,1.288,1.288,1.288,1.315,1.315,1.315,1.315,1.117,1.117,1.117,1.117,1.106,1.247,1.262,1.040
4,0.970,1.006,0.967,0.996,0.987,1.683,0.759,1,0,0,7,1,2020,1,7,1,1,0.985,0.985,0.985,0.985,0.045,0.045,0.045,0.045,0.988,0.988,0.988,0.988,1.294,1.294,1.294,1.294,1.304,1.304,1.304,1.304,1.135,1.135,1.135,1.135,1.058,1.177,1.161,0.962
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
976,1.031,1.034,0.996,1.005,1.013,1.429,0.996,0,1,0,12,2,2024,0,43,1,3,1.065,1.163,1.247,1.269,0.045,0.047,0.043,0.044,1.076,1.173,1.255,1.275,1.419,1.193,1.038,1.114,1.527,1.360,1.269,1.411,1.347,1.133,0.935,0.924,1.016,NaN,NaN,NaN
977,1.010,1.018,0.976,1.008,0.998,1.444,0.976,0,1,0,13,2,2024,1,44,1,1,1.053,1.154,1.247,1.278,0.044,0.048,0.043,0.045,1.062,1.163,1.254,1.284,1.037,0.976,0.794,0.894,1.113,1.124,0.977,1.156,0.881,0.804,0.632,0.637,0.999,NaN,NaN,NaN
978,1.009,1.018,0.995,0.999,1.005,1.456,0.985,0,0,0,14,2,2024,2,45,1,1,1.047,1.145,1.244,1.285,0.036,0.046,0.043,0.045,1.054,1.153,1.251,1.291,1.161,1.290,1.010,1.179,1.221,1.471,1.226,1.522,0.967,1.008,0.781,0.796,NaN,NaN,NaN,NaN
979,0.972,1.015,0.969,1.000,1.000,1.407,0.952,1,0,0,15,2,2024,3,46,1,1,1.004,1.090,1.190,1.239,0.037,0.044,0.042,0.044,1.009,1.098,1.197,1.244,0.715,0.864,0.659,0.761,0.720,0.946,0.769,0.950,0.651,0.691,0.525,0.524,NaN,NaN,NaN,NaN
